In [ ]:
# make sure to install these packages before running:
!pip install sodapy

import pandas as pd
from sodapy import Socrata
import requests
import json

In [ ]:
#In this cell, we include the functions that we might need to use as we try to run MLR.

#Read Data
def get_data(records = 2000):
  import pandas as pd
  from sodapy import Socrata
  import requests
  import json

  # Unauthenticated client only works with public data sets. Note 'None'
  # in place of application token, and no username or password:
  client = Socrata("opendata.utah.gov", 'GkoHUxXZ1JrjaFB0FlLHpniwf')

  # First 2000 results, returned as JSON from API / converted to Python list of
  # dictionaries by sodapy.
  results = client.get("herb-zqda", limit = records)

  # Convert to pandas DataFrame
  df = pd.DataFrame.from_records(results)

  return df

# Handle missing data

def drop_columns_missing_data(df, percent = 0.5):
  for col in df:
    if (df[col].isna().sum() / len(df)) > percent:
      df.drop(columns = [col], inplace = True)

  return df

In [ ]:
#This will collect all of the data into a dataframe that we will alter below
df_all = get_data(252500)

In [ ]:
#First we set df = df_all. This helps us if we don't want to get the data from the API everytime, allowing for faster subsequent queries.
df = df_all

In [ ]:
from datetime import datetime

#This creates a new column, generated from crash_datetime that shows the time of day that the crash occurred
df['crash_time'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.hour

#This creates a new column, generated from crash_datetime that shows the day of the month that the crash occurred
df['crash_day'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.day

#This creates a new column, generated from crash_datetime that shows the day of the week that the crash occurred
df['crash_dayofweek'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.dayofweek

#This creates a new column, generated from crash_datetime that shows the year that the crash occurred
df['crash_year'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.year

#This creates a new column, generated from crash_datetime that shows the month that the crash occurred
df['crash_month'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.month

In [ ]:

from sqlalchemy import create_engine
import urllib.parse
urllib.parse.quote_plus("http://aa1zdq2gijc4ui.c1dtnhbcknoc.us-east-1.rds.amazonaws.com/")
!pip install mysql-connector-python

     |████████████████████████████████| 37.6 MB 1.3 MB/s 


In [ ]:
#Here we drop any columns that we won't be using as part of our regression (mainly ones that will not work as dummy codes by nature)
df = df.drop(columns=['main_road_name','long_utm_x','lat_utm_y','milepoint','route', 'crash_id', 'crash_day', 'crash_datetime', 'city', 'county_name'])

In [ ]:
#The first function called drops any columns with more than 50% of it's data missing (this is not important needed here, since no columns are missing 
#that much data, but it could be useful with future datasets). The second function drops any rows in which the crash_severity_id (which will be the 
#label) is missing.

df = drop_columns_missing_data(df)
df = df.dropna(subset=['crash_severity_id'])
df = df.dropna(subset=['work_zone_related'])

In [ ]:
#This is helpful to show the distrubution of crashes in the dataset by varying levels of severity

df['crash_severity_id'].value_counts()

1    177189
2     43871
3     23889
4      4652
5       960
Name: crash_severity_id, dtype: int64

In [ ]:
#For any other variable that has missing values (in this case it is only the work_zone_related field), this guesses the variable for that column. It
#also creates dummy codes for the other non-numeric variables. This also changes crash severity id to a float, so we need to make sure that is changed back.
#This is done in the next cell down
#df = impute_mean(df, 'crash_severity_id')

for col in df:
  if not pd.api.types.is_numeric_dtype(df[col]) and col != 'crash_severity_id':
    df = pd.get_dummies(df, columns=[col], drop_first=True)

In [ ]:
#Imports decision tree tools from sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Split dataset in features and target variable

y = df['crash_severity_id'] # Label
X = df.drop(columns=['crash_severity_id']) # Features

# Split dataset into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

# Predict the labels for test dataset
y_pred = clf.predict(X_test)

# View the predicted versus actual in a DataFrame
output_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred,})
print(output_df.head(10))

print(output_df['Predicted'].value_counts())
print(output_df['Actual'].value_counts())

       Actual Predicted
252458      4         2
162436      1         2
107983      1         1
119121      1         1
136778      1         1
221811      3         1
105312      3         1
130930      1         1
195936      1         3
43687       4         2
1    59779
2     8829
3     5110
4     1154
5      297
Name: Predicted, dtype: int64
1    53296
2    13138
3     7042
4     1398
5      295
Name: Actual, dtype: int64


In [ ]:
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 12345)

clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f"Accuracy:\t{metrics.accuracy_score(y_test, y_pred)}")

Accuracy:	0.6341177879178916


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.75      0.84      0.79     53204
           2       0.20      0.14      0.16     13010
           3       0.22      0.15      0.18      7253
           4       0.12      0.11      0.11      1373
           5       0.11      0.09      0.10       329

    accuracy                           0.63     75169
   macro avg       0.28      0.26      0.27     75169
weighted avg       0.59      0.63      0.61     75169



In [ ]:
from sklearn.metrics import roc_auc_score

# Generate class membership probabilities
y_preb_probs = clf.predict_proba(X_test)

roc_auc_score(
    y_test, y_preb_probs, average="weighted", multi_class="ovr"
)

0.5658563016574645

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_test, y_pred)

0.1083563768505148

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_corrcoef(y_test, y_pred)

0.11090578850799501

In [ ]:
from sklearn.metrics import log_loss

log_loss(y_test, y_preb_probs)

10.339658569318308

In [ ]:
new_test = [
            [10.0, 4.0, 2019.0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
            [22.0, 0.0, 2016.0, 12.0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
            [12.0, 3.0, 2019.0, 2.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]

new_prediction = clf.predict(new_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
print(new_prediction)

['1' '4' '5']


In [ ]:
!pip install skl2onnx
!pip install onnxruntime

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt
import numpy as np

In [ ]:
#Converts all dtypes to type float
for col in df:
  if col != 'crash_severity_id':
    df[col] = df[col].astype(float)

In [ ]:
ONNXModelPath = "../crash_model.onnx"

num_features = 23

initial_type = [('float_input', FloatTensorType([None, num_features]))]
onnx = convert_sklearn(clf, initial_types=initial_type)
with open(ONNXModelPath, "wb") as f:
    f.write(onnx.SerializeToString())

In [ ]:
sess = rt.InferenceSession(ONNXModelPath)
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run(None, {input_name: X_train.values.astype(np.float32)})[0]
print(pred_onx)

['1' '1' '1' ... '3' '2' '1']


In [ ]:
NewValue = np.array([[12.0, 3.0, 2019.0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
sess.run(None, {input_name: NewValue.astype(np.float32)})

[array(['1'], dtype=object),
 [{'1': 0.6000000238418579,
   '2': 0.20000000298023224,
   '3': 0.20000000298023224,
   '4': 0.0,
   '5': 0.0}]]